In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
# ── Paths (edit if your folder name differs) ───────────────────────────────────
ROOT = "/content/drive/MyDrive/celeba_dataset"
SRC_DIR = f"{ROOT}/Celebrity Image Subsets"    # contains images__<ID>_ folders
COMP_ROOT = f"{ROOT}/CompositeDataset"         # will be created

IMAGES_DIR = f"{COMP_ROOT}/images"
LABELS_DIR = f"{COMP_ROOT}/labels"
META_DIR   = f"{COMP_ROOT}/meta"

# ── Create folders ─────────────────────────────────────────────────────────────
import os, shutil, random, json, math, glob
from pathlib import Path

for p in [COMP_ROOT, IMAGES_DIR, LABELS_DIR, META_DIR]:
    os.makedirs(p, exist_ok=True)
for split in ["train","val","test"]:
    os.makedirs(f"{IMAGES_DIR}/{split}", exist_ok=True)
    os.makedirs(f"{LABELS_DIR}/{split}", exist_ok=True)

# ── Install deps ───────────────────────────────────────────────────────────────
!pip -q install ultralytics==8.3.34 opencv-python pillow numpy tqdm

import cv2
import numpy as np
from PIL import Image, ImageEnhance, ImageFilter
from tqdm.auto import tqdm
random.seed(42)
np.random.seed(42)


In [ ]:
# ── Parse celeb IDs from folder names images__<ID>_ ────────────────────────────
folders = sorted([p for p in glob.glob(f"{SRC_DIR}/images__*_") if os.path.isdir(p)])
def parse_id(path):
    # expects .../images__<ID>_/
    base = os.path.basename(path.strip("/"))

    try:
        return base.split("__")[1].rstrip("_")
    except Exception:
        return None

raw_ids = [parse_id(p) for p in folders]
celeb_ids = [cid for cid in raw_ids if cid is not None]
assert len(celeb_ids) >= 2, "No celebrity folders found. Check naming pattern."

# Ensure unique & stable order (sorted as strings keeps IDs like 3401 intact)
celeb_ids = sorted(list(set(celeb_ids)), key=lambda x: int(x))
num_classes = len(celeb_ids)
print(f"Found {num_classes} celebrity IDs.")

# ── Build id_map.csv (idx → celeb_id) ─────────────────────────────────────────
import csv
with open(f"{META_DIR}/id_map.csv", "w", newline="") as f:
    writer = csv.writer(f)
    writer.writerow(["idx","celeb_id"])
    for idx, cid in enumerate(celeb_ids):
        writer.writerow([idx, cid])

# Helper: map celeb_id -> class_idx
id_to_idx = {cid: i for i, cid in enumerate(celeb_ids)}


In [ ]:

GRID_ROWS, GRID_COLS = 4, 4     # 4×4 grid (16 cells)
CANVAS = (1024, 1024)           # larger image so faces stay visible
CELL_W = CANVAS[0] // GRID_COLS
CELL_H = CANVAS[1] // GRID_ROWS
GUTTER = 8                      # slightly smaller gaps

N_BASE = 1600
SPLIT_COUNTS = {"train": 1120, "val": 240, "test": 240}


# Gather image lists per celeb_id
id_to_images = {}
valid_exts = {".jpg",".jpeg",".png",".bmp"}
for cid in celeb_ids:
    folder = f"{SRC_DIR}/images__{cid}_"
    imgs = []
    for ext in valid_exts:
        imgs.extend(glob.glob(f"{folder}/**/*{ext}", recursive=True))
    imgs = [p for p in imgs if os.path.isfile(p)]
    if len(imgs) == 0:
        print(f"Warning: no images for celeb {cid}")
    id_to_images[cid] = imgs

# Round-robin sampler to keep balance across 45 celebs
from itertools import cycle
rr_ids = []
repeat = math.ceil((N_BASE * GRID_ROWS * GRID_COLS) / len(celeb_ids))  # total slots / num IDs
balanced_list = celeb_ids * repeat
random.shuffle(balanced_list)
rr = cycle(balanced_list)


In [ ]:
def fit_within(img: Image.Image, cell_w, cell_h, keep_aspect=True):
    """Resize PIL image to fit within cell while keeping aspect; returns resized image."""
    if keep_aspect:
        img = img.copy()
        img.thumbnail((cell_w - GUTTER, cell_h - GUTTER), Image.LANCZOS)
        return img
    else:
        return img.resize((cell_w - GUTTER, cell_h - GUTTER), Image.LANCZOS)

def cell_origin(col, row, cell_w, cell_h, gutter=0):
    x0 = col*cell_w + gutter//2
    y0 = row*cell_h + gutter//2
    return x0, y0

def paste_center(canvas: Image.Image, tile: Image.Image, x0, y0, cell_w, cell_h):
    """Paste tile centered within the cell at (x0,y0). Returns (xmin,ymin,xmax,ymax) on canvas."""
    tw, th = tile.size
    cx = x0 + cell_w//2
    cy = y0 + cell_h//2
    xmin = int(cx - tw//2)
    ymin = int(cy - th//2)
    xmin = max(xmin, x0)
    ymin = max(ymin, y0)
    xmax = min(xmin + tw, x0 + cell_w - 1)
    ymax = min(ymin + th, y0 + cell_h - 1)
    canvas.paste(tile, (xmin, ymin))
    return xmin, ymin, xmax, ymax

def xyxy_to_yolo(xmin, ymin, xmax, ymax, W, H):
    w = max(1, xmax - xmin + 1)
    h = max(1, ymax - ymin + 1)
    cx = xmin + w/2
    cy = ymin + h/2
    return cx/W, cy/H, w/W, h/H

def rand_pick_image(cid):
    imgs = id_to_images.get(cid, [])
    if not imgs:
        return None
    return random.choice(imgs)


In [ ]:
def next_n_distinct(rr_iter, pool, n):
    """Return n distinct celeb IDs with available images."""
    chosen = set()
    attempt = 0
    while len(chosen) < n and attempt < 2000:
        c = next(rr_iter)
        if c in pool and id_to_images.get(c):
            chosen.add(c)
        attempt += 1
    return list(chosen)

counter = {"train": 0, "val": 0, "test": 0}
index = 0

for split in ["train", "val", "test"]:
    target = SPLIT_COUNTS[split]
    pbar = tqdm(total=target, desc=f"Building {split} composites")
    built = 0
    while built < target:
        # choose distinct celeb IDs for a 4x4 grid (16 total)
        num_needed = GRID_ROWS * GRID_COLS   # = 16 if 4x4
        cids = next_n_distinct(rr, celeb_ids, num_needed)
        if len(cids) < num_needed:
            print(f"Warning: Only found {len(cids)} distinct celeb IDs for a {GRID_ROWS}x{GRID_COLS} grid. Skipping.")
            continue

        canvas = Image.new("RGB", CANVAS, (245, 245, 245))
        labels = []
        ok = True
        used = set()

        # place tiles row-major
        for r in range(GRID_ROWS):
            for c in range(GRID_COLS):
                cid = cids[r * GRID_COLS + c]
                if cid in used:
                    ok = False
                    break
                used.add(cid)

                src_path = rand_pick_image(cid)
                if src_path is None:
                    ok = False
                    break

                try:
                    tile = Image.open(src_path).convert("RGB")
                except:
                    ok = False
                    break

                tile = fit_within(tile, CELL_W, CELL_H, keep_aspect=True)
                x0, y0 = cell_origin(c, r, CELL_W, CELL_H, GUTTER)
                xmin, ymin, xmax, ymax = paste_center(canvas, tile, x0, y0, CELL_W, CELL_H)
                cx, cy, w, h = xyxy_to_yolo(xmin, ymin, xmax, ymax, CANVAS[0], CANVAS[1])

                class_idx = id_to_idx[cid]
                labels.append((class_idx, cx, cy, w, h))
            if not ok:
                break

        # skip if image incomplete
        if not ok or len(labels) != num_needed:
            continue

        stem = f"comp_{index:06d}"
        img_out = f"{IMAGES_DIR}/{split}/{stem}.jpg"
        lab_out = f"{LABELS_DIR}/{split}/{stem}.txt"

        canvas.save(img_out, quality=92)
        with open(lab_out, "w") as f:
            for cls, cx, cy, w, h in labels:
                f.write(f"{cls} {cx:.6f} {cy:.6f} {w:.6f} {h:.6f}\n")

        built += 1
        index += 1
        pbar.update(1)
    pbar.close()


In [ ]:
AUG_TRAIN = True

def augment_photometric(pil_img):
    # mild brightness & contrast jitter, occasional blur
    br = ImageEnhance.Brightness(pil_img).enhance( np.random.uniform(0.9, 1.1) )
    ct = ImageEnhance.Contrast(br).enhance( np.random.uniform(0.9, 1.1) )
    if np.random.rand() < 0.25:
        ct = ct.filter(ImageFilter.GaussianBlur(radius=np.random.uniform(0.2, 0.8)))
    return ct

def flip_labels_horiz(yolo_lines):
    out = []
    for line in yolo_lines:
        cls, cx, cy, w, h = line
        cx_flipped = 1.0 - cx
        out.append((cls, cx_flipped, cy, w, h))
    return out

if AUG_TRAIN:
    train_imgs = sorted(glob.glob(f"{IMAGES_DIR}/train/*.jpg"))
    for img_path in tqdm(train_imgs, desc="Augmenting train (+1x)"):
        stem = Path(img_path).stem
        lab_path = f"{LABELS_DIR}/train/{stem}.txt"
        if not os.path.exists(lab_path):
            continue

        # read labels
        yolo_lines = []
        with open(lab_path, "r") as f:
            for ln in f:
                toks = ln.strip().split()
                cls = int(toks[0]); cx,cy,w,h = map(float, toks[1:5])
                yolo_lines.append((cls,cx,cy,w,h))

        # 50% flip horizontally
        img = Image.open(img_path).convert("RGB")
        do_flip = np.random.rand() < 0.5
        if do_flip:
            img_aug = img.transpose(Image.FLIP_LEFT_RIGHT)
            labels_aug = flip_labels_horiz(yolo_lines)
        else:
            img_aug = img.copy()
            labels_aug = yolo_lines

        # photometric
        img_aug = augment_photometric(img_aug)

        # save with new stem
        new_stem = stem + "_aug1"
        img_out = f"{IMAGES_DIR}/train/{new_stem}.jpg"
        lab_out = f"{LABELS_DIR}/train/{new_stem}.txt"

        img_aug.save(img_out, quality=92)
        with open(lab_out, "w") as f:
            for (cls,cx,cy,w,h) in labels_aug:
                f.write(f"{cls} {cx:.6f} {cy:.6f} {w:.6f} {h:.6f}\n")


In [ ]:
data_yaml = {
    "path": COMP_ROOT,
    "train": "images/train",
    "val":   "images/val",
    "test":  "images/test",
    "nc":    num_classes,
    "names": [str(cid) for cid in celeb_ids]  # <- true celeb IDs as strings
}
import yaml
with open(f"{COMP_ROOT}/data.yaml", "w") as f:
    yaml.safe_dump(data_yaml, f, sort_keys=False)

# Record build meta for reproducibility
build_log = {
    "grid": {"rows": GRID_ROWS, "cols": GRID_COLS, "canvas": CANVAS, "gutter": GUTTER},
    "counts": {"base_total": N_BASE, **SPLIT_COUNTS},
    "aug": {"train_plus_one": bool(AUG_TRAIN), "ops": ["flip(0.5)", "brightness/contrast", "blur(0.25)"]},
    "classes": num_classes,
    "note": "Label class_idx follows meta/id_map.csv; names[] are true celeb IDs."
}
with open(f"{META_DIR}/build_log.json","w") as f:
    json.dump(build_log, f, indent=2)


In [ ]:
from ultralytics import YOLO

data_path = f"{COMP_ROOT}/data.yaml"  # same as before
model = YOLO("yolov8n.pt")

model.train(
    data=data_path,
    imgsz=1024,
    epochs=100,
    batch=16,
    device=0,
    mosaic=1.0,         # spatial: combines random images
    mixup=0.5,          # spatial + visual: blends images
    copy_paste=0.5,     # spatial: pastes objects randomly (overlaps)
    hsv_h=0.015, hsv_s=0.7, hsv_v=0.4,  # visual color jitter
    degrees=5.0,        # small rotation (optional)
    translate=0.1,      # move objects around (optional)
    scale=0.5,          # random zoom in/out (optional)
    shear=2.0,          # slight skew (optional)
    project=f"{COMP_ROOT}/runs",
    name="yolov8n_grid4x4_spatial_aug"
)


In [ ]:
from ultralytics import YOLO
import pandas as pd

# Load best weights
runs_dir = f"{COMP_ROOT}/runs/yolov8n_grid4x4_spatial_aug/weights"
best = f"{runs_dir}/best.pt"

predictor = YOLO(best)
test_images = sorted(glob.glob(f"{IMAGES_DIR}/test/*.jpg"))[:12]  # a few samples

results = predictor.predict(
    source=test_images,
    imgsz=640,
    conf=0.25,
    save=True,                      # saves images with boxes
   project=f"{COMP_ROOT}/preds",
name="test_samples_4x4",
    verbose=False
)

# Pretty print: Celeb ID, conf, bbox (xyxy)
for r, img_path in zip(results, test_images):
    print(f"\nImage: {Path(img_path).name}")
    if r.boxes is None or len(r.boxes) == 0:
        print("  No detections.")
        continue
    ids = r.boxes.cls.cpu().numpy().astype(int)
    confs = r.boxes.conf.cpu().numpy()
    xyxy = r.boxes.xyxy.cpu().numpy()

    rows = []
    for i, c in enumerate(ids):
        celeb_id_str = data_yaml["names"][c]   # true celeb ID string
        x1,y1,x2,y2 = xyxy[i]
        rows.append([celeb_id_str, float(confs[i]), (int(x1),int(y1),int(x2),int(y2))])
    df = pd.DataFrame(rows, columns=["Celeb ID","Confidence","Bbox (x_min,y_min,x_max,y_max)"])
    display(df)

In [ ]:
# --- Simple Object Detection Demo ---
from ultralytics import YOLO
import yaml
import matplotlib.pyplot as plt
from google.colab import files

# Load model and label info
COMP_ROOT = "/content/drive/MyDrive/celeba_dataset/CompositeDataset"
MODEL_PATH = f"{COMP_ROOT}/runs/yolov8n_grid4x4_spatial_aug/weights/best.pt"
DATA_YAML = f"{COMP_ROOT}/data.yaml"

model = YOLO(MODEL_PATH)
with open(DATA_YAML, "r") as f:
    names = yaml.safe_load(f)["names"]

# Upload input image (professor gives in class)
uploaded = files.upload()
img_path = list(uploaded.keys())[0]

# Run detection
res = model(img_path, conf=0.25, imgsz=1024)[0]

# Display result with bounding boxes
annotated = res.plot()
plt.figure(figsize=(8,8))
plt.imshow(annotated[..., ::-1])
plt.axis("off")
plt.title("Detected Celebrities (IDs & Locations)")
plt.show()

# Print detections (ID, confidence, box)
print("\nPredictions:")
for i, cid in enumerate(res.boxes.cls.cpu().numpy().astype(int)):
    conf = float(res.boxes.conf[i])
    x1, y1, x2, y2 = res.boxes.xyxy[i].cpu().numpy()
    print(f"  ID: {names[cid]} | Conf: {conf:.2f} | Box: ({int(x1)}, {int(y1)}, {int(x2)}, {int(y2)})")


In [ ]:
!pip -q install ultralytics==8.3.34 opencv-python pillow numpy tqdm

In [ ]:
!git add .